# Contrastive Knowledge Assesment (CKA) Notebook Demo
This notebook enables interactive experimentation with CKA for models including `Flan-t5s`, `OPTs`, `Robertas`, and `GPT2s`.
The goal is to probe if factual statements are predicted at a higher probability than a given counterfactual.

<a target="_blank" href="https://colab.research.google.com/github/daniel-furman/Capstone/blob/main/notebooks/cka_run_main_demo.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


## Dependencies

In [ ]:
!git clone https://github.com/daniel-furman/Capstone.git
!pip install -r /content/Capstone/requirements.txt

Cloning into 'Capstone'...
remote: Enumerating objects: 556, done.
remote: Counting objects: 100% (213/213), done.
remote: Compressing objects: 100% (127/127), done.
remote: Total 556 (delta 114), reused 155 (delta 60), pack-reused 343
Receiving objects: 100% (556/556), 24.37 MiB | 17.05 MiB/s, done.
Resolving deltas: 100% (276/276), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 49.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.3/76.3 MB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 91.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 4.5 MB/s eta 0:00:00


## Imports

In [ ]:
import os

In [ ]:
os.chdir('/content/Capstone/src/cka_scripts')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

## Notebook usage

In [ ]:
# import the main wrapper function for running cka

from run_cka import main
config = {}

Here, you can specify a causal, masked, or text2text large language model in ```config["models"]```. The `transformers` package is used to load the model into the session at mixed 8bit precision (`load_in_8_bit=True`). See [README](https://github.com/daniel-furman/Capstone#models-tested) for the full list of model families supported and HuggingFace for specific model tags.  

Some example VRAM usages:
* `google/flan-ul2` requires slightly more than 32GB VRAM
* `EleutherAI/gpt-neox-20b` requires slightly more than 22GB VRAM
* `EleutherAI/gpt-j-6B` requires slightly more than 7GB VRAM
* `google/flan-t5-xl`requires slightly more than 5GB VRAM
* `roberta-large` requires slightly less than 1GB VRAM



In [ ]:
config["models"] = [
    "EleutherAI/gpt-j-6B",
    #"gpt2-xl",
]

Next, new facts can be input in ```config["input_information"]```.
  * For instance: to input "Lebron James is famous for playing the sport of {true: basketball; false: football}", see the last example. 

In [ ]:
config["input_information"] = {
    
        "0": {
            "stem": "The 2020 Olympics were held in",
            "true": "Tokyo",
            "false": ["London", "Berlin", "Chicago"],
        },
        "1": {
            "stem": "Operation Overlord took place in",
            "true": "Normandy",
            "false": ["Manila", "Santiago", "Baghdad"],
        },
        "2": {
            "stem": "Steve Jobs is the founder of",
            "true": "Apple",
            "false": ["Microsoft", "Google", "Facebook"],
        },

        # Example addition(s)
        "3": {
            "stem": "Lebron James is famous for playing the sport of",
            "true": "basketball",
            "false": ["football"],
        },   
    }

Lastly, verbosity controls how much info is printed to the user.

In [ ]:
config["verbosity"] = False

In [ ]:
config

{'models': ['google/flan-ul2'],
 'input_information': {'0': {'stem': 'The 2020 Olympics were held in',
   'true': 'Tokyo',
   'false': ['London', 'Berlin', 'Chicago']},
  '1': {'stem': 'Operation Overlord took place in',
   'true': 'Normandy',
   'false': ['Manila', 'Santiago', 'Baghdad']},
  '2': {'stem': 'Steve Jobs is the founder of',
   'true': 'Apple',
   'false': ['Microsoft', 'Google', 'Facebook']},
  '3': {'stem': 'Lebron James is famous for playing the sport of',
   'true': 'basketball',
   'false': ['football']}},
 'verbosity': False}

In [ ]:
score_dicts = main(config)

print(score_dicts[0])
print(score_dicts[1])
print(score_dicts[2])

CKA for google/flan-ul2
Loading  model...



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Running comparisons...


100%|██████████| 4/4 [00:11<00:00,  2.92s/it]

Done

{'google/flan-ul2': [{"The 2020 Olympics were held in ['Tokyo', 'London']": {'p_true': 0.75732421875, 'p_false': 0.00010901689529418945, 'p_true - p_false': 0.7572152018547058, 'p_true > p_false': 'True', 'p_true / (p_true + p_false)': 0.9998560574203456}}, {"The 2020 Olympics were held in ['Tokyo', 'Berlin']": {'p_true': 0.75732421875, 'p_false': 2.8014183044433594e-06, 'p_true - p_false': 0.7573214173316956, 'p_true > p_false': 'True', 'p_true / (p_true + p_false)': 0.999996287709018}}, {"The 2020 Olympics were held in ['Tokyo', 'Chicago']": {'p_true': 0.75732421875, 'p_false': 1.3232231140136719e-05, 'p_true - p_false': 0.7573109865188599, 'p_true > p_false': 'True', 'p_true / (p_true + p_false)': 0.9999825147548848}}, {"Operation Overlord took place in ['Normandy', 'Manila']": {'p_true': 0.54541015625, 'p_false': 5.960464477539063e-08, 'p_true - p_false': 0.5454100966453552, 'p_true > p_false': 'True', 'p_true / (p_true + p_false)': 0.9999998723811151}}, {"Operation Overlord 

## CLI usage 
* Compatible with cached configs included in the repo
* You can also create a custom config for an experiment
    * See: ```./content/Capstone/src/cka_scripts/configs```

In [ ]:
# full benchmark dataset

!python run_cka.py configs.rome_full.distilgpt2_rome_full

CKA for distilgpt2
Loading  model...

===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
Running comparisons...
100% 21919/21919 [10:21<00:00, 35.26it/s]
Done


Score dict summary:
{'distilgpt2': 'This model predicted 15099/21919 facts at a higher prob than the given counterfactual. The mean p_true / (p_true + p_false) was 0.6568 while the mean p_true was 0.0095'}
